# NFL Scores and Betting Data

## First Look

The primary data we will be working with comes from https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data

Before we can get into analysis and work towards answering our questions, we need to get an idea of what our data looks like. 

In [1]:
# Dependencies and Setup
import pandas as pd

# Disable settingwithcopy wanrning - doesn't like chaining index modifications on the same dataframe
# Will not be an issue for this project, but it's usually not a good idea to disable the warning like this
pd.set_option('mode.chained_assignment', None)

In [2]:
# load csvs into dataframes
# stadium csv has an encoding issue, will resolve to fix but data is not vital - skipping for now
# stadiums_df = pd.read_csv("raw data/nfl-scores-and-betting-data/nfl_stadiums.csv") 
teams_df = pd.read_csv("raw data/nfl-scores-and-betting-data/nfl_teams.csv")
scorelines_df = pd.read_csv("raw data/nfl-scores-and-betting-data/spreadspoke_scores.csv")

The most useful and interesting data is located in the scorelines dataframe, so let's get a look at it

In [3]:
scorelines_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


Now we want to find out how big this dataframe is. There are a number of ways to do this, such as doing a describe on the dataframe or simply doing len(scorelines_df). However, we also want to get a look at the data near the end of the dataframe to see how it comapres to the first rows. 

Looking at the tail also gives us a quick look at how recent the data is. In this case, the data is very current and goes all the way up to the most recent football games played (the Superbowl).

In [4]:
scorelines_df.tail()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
12673,01/12/2020,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,GB,-4.5,45.5,Lambeau Field,False,NaN,NaN,NaN,NaN
12674,01/12/2020,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,KC,-10.0,50.5,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12675,01/19/2020,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,KC,-7.0,51,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12676,01/19/2020,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,SF,-8.0,46.5,Levi's Stadium,False,NaN,NaN,NaN,NaN
12677,02/02/2020,2019,Superbowl,True,Kansas City Chiefs,31,20,San Francisco 49ers,KC,-1.5,53,Hard Rock Stadium,True,NaN,NaN,NaN,NaN


12,677 rows is a lot of data. While it all could be useful, it's best to start with more manageable data and work our way outwards. We can also see the weather data has some NaN values that we will need to take care of later.

## Cleaning

First, we'll start by trimming the dataframe to only contain data on games played in roughly the last 20 years (year 2000 and beyond).

In [5]:
scorelines_trimmed_df = scorelines_df[scorelines_df['schedule_season'] >= 2000]

In [6]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
7354,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False,72.0,0.0,NaN,DOME
7355,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False,70.0,7.0,82,NaN
7356,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False,75.0,8.0,86,NaN
7357,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False,95.0,7.0,32,NaN
7358,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False,69.0,13.0,87,NaN


Looks good, but now the index is way off. We can't really tell where we are. We'll reset it to account for our smaller dataframe and keep it inplace so that it modifies the trimmed dataframe. 

We'll also drop the old dataframe index rather than add it. It doesn't really tell us anything meaningful that we could later use.

In [7]:
scorelines_trimmed_df.reset_index(drop=True, inplace=True)

In [8]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False,72.0,0.0,NaN,DOME
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False,70.0,7.0,82,NaN
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False,75.0,8.0,86,NaN
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False,95.0,7.0,32,NaN
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False,69.0,13.0,87,NaN


In [9]:
len(scorelines_trimmed_df)

5324

The index has been reset and we now have a dataframe that has 5324 rows. Still a good amount of data, but much more manageable.

Now let's deal with the weather data. Looking at it could provide some interesting comparisons and trends, but the fact that data is missing is a problem. It's also not something we directly wanted to look at, so for now it's best to just remove the data from the dataframe. If this project were going to continue in the future, the weather data would be a good item to return to and work on more closely.

In [10]:
scorelines_trimmed_df.drop(['weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail'], axis=1, inplace=True)

In [11]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False


Now we have a full dataframe complete with data that will be useful to us.

Looking at it, however, we can see that the team favorite id doesn't match the team names. Sure, it tells us who the favorite is and anyone reading the data can compare and determine which team is represented by the abbreviation. But the computer can't do that. If we want to run comparisons using the favorite team the computer needs to know which team those abbreviations mean.

We could address this a number of ways at different points in the procress. For example, we could implement a comparison dictionary later on that just substitutes the full name in place of the abbreviation when we need it. However, we feel it will be easier if we just use that dictionary to replace the favorite id with the full team name for now.

In [12]:
# First we need to see all the possible abbreviations.
scorelines_trimmed_df['team_favorite_id'].unique()

array(['ATL', 'BUF', 'JAX', 'DAL', 'GB', 'IND', 'MIA', 'MIN', 'TB',
       'PICK', 'NYG', 'OAK', 'BAL', 'WAS', 'LAR', 'ARI', 'CIN', 'DEN',
       'PHI', 'LAC', 'CAR', 'TEN', 'NYJ', 'CHI', 'PIT', 'KC', 'NE', 'SEA',
       'NO', 'DET', 'SF', 'CLE', 'HOU'], dtype=object)

In [13]:
# Now we need to see all the possible team names.
scorelines_trimmed_df['team_home'].unique()

array(['Atlanta Falcons', 'Buffalo Bills', 'Cleveland Browns',
       'Dallas Cowboys', 'Green Bay Packers', 'Kansas City Chiefs',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'Oakland Raiders',
       'Pittsburgh Steelers', 'Washington Redskins', 'St. Louis Rams',
       'Arizona Cardinals', 'Baltimore Ravens', 'Cincinnati Bengals',
       'Denver Broncos', 'Detroit Lions', 'Indianapolis Colts',
       'Philadelphia Eagles', 'San Diego Chargers', 'San Francisco 49ers',
       'Seattle Seahawks', 'Tampa Bay Buccaneers', 'Tennessee Titans',
       'New York Jets', 'Carolina Panthers', 'Chicago Bears',
       'Jacksonville Jaguars', 'Houston Texans', 'Los Angeles Rams',
       'Los Angeles Chargers'], dtype=object)

In [14]:
# With the info from the above two cells, now it's a matter of matching the correct abbreviation to the correct team in a dictionary.
team_names = {'ATL': 'Atlanta Falcons', 'BUF': 'Buffalo Bills', 'JAX': 'Jacksonville Jaguars', 'DAL': 'Dallas Cowboys', 'GB': 'Green Bay Packers', 'IND': 'Indianapolis Colts', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings', 'TB': 'Tampa Bay Buccaneers', 'NYG': 'New York Giants', 'OAK': 'Oakland Raiders', 'BAL': 'Baltimore Ravens', 'ARI': 'Arizona Cardinals', 'CIN': 'Cincinnati Bengals', 'DEN': 'Denver Broncos', 'PHI': 'Philadelphia Eagles', 'LAC': 'Los Angeles Chargers', 'CAR': 'Carolina Panthers', 'TEN': 'Tennessee Titans', 'NYJ': 'New York Jets', 'CHI': 'Chicago Bears', 'PIT': 'Pittsburgh Steelers', 'KC': 'Kansas City Chiefs', 'NE': 'New England Patriots', 'SEA': 'Seattle Seahawks', 'NO': 'New Orleans Saints', 'DET': 'Detroit Lions', 'SF': 'San Francisco 49ers', 'CLE': 'Cleveland Browns', 'HOU': 'Houston Texans', 'WAS': 'Washington Redskins', 'LAR': 'Los Angeles Rams'}

In [15]:
# Now we use the dictionary to replace the abbreviations in the entire id column with the full team names
scorelines_trimmed_df.replace({"team_favorite_id": team_names}, inplace=True) 

We're not quite done yet, as we noticed that the team favorite id mysteriously excludes the St. Louis Rams and the San Diego Chargers despite them both being present in the home and away team data.

This likely has something to do with the fact that the Rams moved from St. Louis to Los Angeles in 2016 followed by the Chargers one year later. As such, we'll treat them as different teams for the purposes of our analysis and we need to add a special case for dealing with them.

In [16]:
# Replace the favorite id with the correct team in each row that those teams are found
for index,row in scorelines_trimmed_df.iterrows():
    if (row['team_home'] == "San Diego Chargers" or row['team_away'] == "San Diego Chargers") and row['team_favorite_id'] == "Los Angeles Chargers":
        scorelines_trimmed_df.loc[index,'team_favorite_id'] = "San Diego Chargers"
    if (row['team_home'] == "St. Louis Rams" or row['team_away'] == "St. Louis Rams") and row['team_favorite_id'] == "Los Angeles Rams":
        scorelines_trimmed_df.loc[index,'team_favorite_id'] = "St. Louis Rams"

In [17]:
# Make sure there are no abbreviations hiding in the dataframe
# NOTE - PICK is still present but should not be dealt with. Indicates a scenario where there is no true favorite.
scorelines_trimmed_df['team_favorite_id'].unique()

array(['Atlanta Falcons', 'Buffalo Bills', 'Jacksonville Jaguars',
       'Dallas Cowboys', 'Green Bay Packers', 'Indianapolis Colts',
       'Miami Dolphins', 'Minnesota Vikings', 'Tampa Bay Buccaneers',
       'PICK', 'New York Giants', 'Oakland Raiders', 'Baltimore Ravens',
       'Washington Redskins', 'St. Louis Rams', 'Arizona Cardinals',
       'Cincinnati Bengals', 'Denver Broncos', 'Philadelphia Eagles',
       'San Diego Chargers', 'Carolina Panthers', 'Tennessee Titans',
       'New York Jets', 'Chicago Bears', 'Pittsburgh Steelers',
       'Kansas City Chiefs', 'New England Patriots', 'Seattle Seahawks',
       'New Orleans Saints', 'Detroit Lions', 'San Francisco 49ers',
       'Cleveland Browns', 'Houston Texans', 'Los Angeles Rams',
       'Los Angeles Chargers'], dtype=object)

## Calculations

Data analysis is a cyclical process, even just within a single analysis.

Now that we have a solid data with most of the data we want, we should make things easier for ourselves in the analysis phase. We'll bounce back and forth from that section to here as we discover more about the data and get a clearer idea of the paths we want to head down.

While the data currently gives us the score and we can easily determine who won and lost, there's no columns that explicitly state the winner or loser. Having one will make comparisons and observations easier in the future. Some information about the winner will also be good, such as whether or not they were home or away and whether or not they were the favorite.

Since we want to focus on information that would be most beneficial to sports bettors, it's important that we have a column determining the result of the over/under line as well as whether or not the favorite team covered the spread (won by more points than the projected difference).

In [18]:
scorelines_trimmed_df['team_win'] = ""
scorelines_trimmed_df['team_loss'] = ""
scorelines_trimmed_df['home_away_win'] = ""
scorelines_trimmed_df['fav_win'] = ""
scorelines_trimmed_df['over_under_result'] = ""
scorelines_trimmed_df['fav_spread_cover'] = ""

In [19]:
# Make sure all columns were added correctly without errors (spelling, etc.)
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,team_win,team_loss,home_away_win,fav_win,over_under_result,fav_spread_cover
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,Atlanta Falcons,-6.5,46.5,Georgia Dome,False,,,,,,
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,Buffalo Bills,-1.0,40,Ralph Wilson Stadium,False,,,,,,
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,Jacksonville Jaguars,-10.5,38.5,FirstEnergy Stadium,False,,,,,,
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,Dallas Cowboys,-6.0,39.5,Texas Stadium,False,,,,,,
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,Green Bay Packers,-2.5,44,Lambeau Field,False,,,,,,


With the data we have, the easiest way to do our calculations will be to iterate over each row and perform a number of comparisons. This will be a bit performance intensive over 5000+ rows, but should be manageable. The for statement could be repeated in different cells if cyclomatic complexity gets too large.

In [20]:
for index, row in scorelines_trimmed_df.iterrows():

    # determine the winning/losing teams and whether or not the winning team was home/away
    if scorelines_trimmed_df.loc[index, 'score_home'] > scorelines_trimmed_df.loc[index, 'score_away']:
        scorelines_trimmed_df.loc[index, 'team_win'] = scorelines_trimmed_df.loc[index, 'team_home']
        scorelines_trimmed_df.loc[index, 'team_loss'] = scorelines_trimmed_df.loc[index, 'team_away']
        scorelines_trimmed_df.loc[index, 'home_away_win'] = "Home"
    else:
        scorelines_trimmed_df.loc[index, 'team_loss'] = scorelines_trimmed_df.loc[index, 'team_home']
        scorelines_trimmed_df.loc[index, 'team_win'] = scorelines_trimmed_df.loc[index, 'team_away']
        scorelines_trimmed_df.loc[index, 'home_away_win'] = "Away"

    # determine if the favorite team won
    if scorelines_trimmed_df.loc[index, 'team_win'] == scorelines_trimmed_df.loc[index, 'team_favorite_id']:
        scorelines_trimmed_df.loc[index, 'fav_win'] = "True"
    else:
        scorelines_trimmed_df.loc[index, 'fav_win'] = "False"

    # determine the result of the over/under bet - sum the scores and compare to the O/U line - larger = over, smaller = under
    if (scorelines_trimmed_df.loc[index, 'score_home'] + scorelines_trimmed_df.loc[index, 'score_away']) > float(scorelines_trimmed_df.loc[index, 'over_under_line']):
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Over"
    elif (scorelines_trimmed_df.loc[index, 'score_home'] + scorelines_trimmed_df.loc[index, 'score_away']) == scorelines_trimmed_df.loc[index, 'over_under_line']:
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Push" # a push happens if the score equals the O/U line, essentially no one wins the bet
    else:
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Under"

    # determine if the favorite team covered the spread - absolute value of the home/away score difference compared to the abs of the spread - larger = cover, smaller = no cover
    if scorelines_trimmed_df.loc[index, 'fav_win'] == 'True': # if the favorite lost, they automatically didn't cover the spread
        if abs(scorelines_trimmed_df.loc[index, 'score_home'] - scorelines_trimmed_df.loc[index, 'score_away']) > abs(float(scorelines_trimmed_df.loc[index, 'spread_favorite'])):
            scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "True"
        else:
            scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "False"
    else:
        scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "False"

In [21]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,team_win,team_loss,home_away_win,fav_win,over_under_result,fav_spread_cover
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,Atlanta Falcons,-6.5,46.5,Georgia Dome,False,Atlanta Falcons,San Francisco 49ers,Home,True,Over,True
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,Buffalo Bills,-1.0,40,Ralph Wilson Stadium,False,Buffalo Bills,Tennessee Titans,Home,True,Under,True
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,Jacksonville Jaguars,-10.5,38.5,FirstEnergy Stadium,False,Jacksonville Jaguars,Cleveland Browns,Away,True,Under,True
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,Dallas Cowboys,-6.0,39.5,Texas Stadium,False,Philadelphia Eagles,Dallas Cowboys,Away,False,Over,False
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,Green Bay Packers,-2.5,44,Lambeau Field,False,New York Jets,Green Bay Packers,Away,False,Under,False


After running all those calculations we check the dataframe to make sure it looks good. No errors have been thrown and our calculations seem to be correct at a glance. We'll determine more once we move fully into the exploratory data analysis phase.

With that completed, it's time to export our cleaned and improved data so that we can analyze it and use it anywhere in our project. Keeping things in csv format seems the most convenient.

In [22]:
scorelines_trimmed_df.to_csv("output data/spreadspoke_scores_trimmed.csv", index=False)

# Further Analysis

Below is an example of a calculation that could be useful in the future. This is data that doesn't work as a new column in the dataframe but examines a trend that would be ideal to graph.

This data could be exported to a json file so it could be shared between notebooks, but it will be easier to just copy the cells in this case. We'll leave the example here as there's no reason to delete it.

The full analysis is in its own notebook, as are the visualizations.

In [23]:
# Initialize an empty dictionary
underdog_wins = {}

In [24]:
# Iterate through the dataframe's rows to find cases where the favorite team didn't win (i.e. the underdog team won)
for index, row in scorelines_trimmed_df.iterrows():
    
    if scorelines_trimmed_df.loc[index, 'fav_win'] == "False":
        if scorelines_trimmed_df.loc[index, 'team_win'] not in underdog_wins: # if the underdog team isn't in the dictionary, add them and initialize their wins at 1
            underdog_wins[scorelines_trimmed_df.loc[index, 'team_win']] = 1
        else: # increment the underdog team's win count by 1
            underdog_wins[scorelines_trimmed_df.loc[index, 'team_win']] += 1 

In [25]:
# The dictionary now contains a count of every team's underdog wins over the past ~20 years.
underdog_wins

{'Philadelphia Eagles': 53,
 'New York Jets': 65,
 'Detroit Lions': 52,
 'Cleveland Browns': 57,
 'Oakland Raiders': 64,
 'New York Giants': 57,
 'New Orleans Saints': 53,
 'Atlanta Falcons': 58,
 'Miami Dolphins': 75,
 'Minnesota Vikings': 57,
 'Denver Broncos': 51,
 'Dallas Cowboys': 51,
 'Green Bay Packers': 45,
 'San Francisco 49ers': 54,
 'Kansas City Chiefs': 56,
 'Washington Redskins': 74,
 'Indianapolis Colts': 44,
 'New England Patriots': 41,
 'Chicago Bears': 62,
 'Pittsburgh Steelers': 51,
 'Baltimore Ravens': 52,
 'Tennessee Titans': 62,
 'Cincinnati Bengals': 62,
 'Jacksonville Jaguars': 67,
 'Arizona Cardinals': 69,
 'Buffalo Bills': 62,
 'Carolina Panthers': 62,
 'Seattle Seahawks': 53,
 'San Diego Chargers': 46,
 'Tampa Bay Buccaneers': 59,
 'Houston Texans': 52,
 'St. Louis Rams': 40,
 'Los Angeles Rams': 7,
 'Los Angeles Chargers': 10}